## Energy Savings App

The Energy Saving Application is a comprehensive, modular pipeline designed to train and deploy a Reinforcement Learning (RL) agent for dynamic network optimization. The core objective is to reduce power consumption by intelligently turning cell sectors on or off and adjusting their antenna tilts based on the time of day, while maintaining acceptable network coverage.

This system leverages a pre-trained Bayesian Digital Twin (BDT) RF model, which allows the RL training loop to perform rapid, local RF simulations. This architecture decouples the RL agent's learning process from the latency of live backend services, enabling efficient training on multi-day traffic patterns.

In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import logging
import os
from IPython.display import Image, display

from apps.energy_savings.data_preprocessor import UEDataPreprocessor
from apps.energy_savings.bdt_manager import BDTManager
from apps.energy_savings.rl_trainer import run_rl_training
from apps.energy_savings.rl_predictor import run_rl_prediction
from apps.energy_savings.energy_saving_visualizer import EnergySavingVisualizer

In [ ]:
# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Setting up Data Directory & Save Locations

*unzip the `data/eneregy_saving_data.zip` file to get `data/energy_saving_data/` folder*

In [ ]:
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

BASE_DATA_DIR = os.path.join(NOTEBOOK_DIR, "data", "energy_saving_data", "generated_data")
STATIC_DATA_DIR = os.path.join(NOTEBOOK_DIR, "data", "energy_saving_data", "data")

In [ ]:
# All relevant data required for this demo are located in the following directories
TOPOLOGY_PATH = os.path.join(STATIC_DATA_DIR, "topology.csv")
CONFIG_PATH = os.path.join(STATIC_DATA_DIR, "config.csv")
TRAINING_DATA_PATH = os.path.join(STATIC_DATA_DIR, "dummy_ue_training_data.csv")

# All data saved during the training and prediction processes are located in the following directories
BDT_MODEL_PATH = os.path.join("data", "energy_saving_data", "bdt_model_map.pickle")
RL_MODEL_PATH = os.path.join("data", "energy_saving_data", "energy_saver_agent.zip")
RL_LOG_DIR = os.path.join("data", "energy_saving_data", "rl_training_logs")
PLOT_OUTPUT_DIR = os.path.join(STATIC_DATA_DIR, "plots")

# Showcasing **Energy Savings** Functions: 

Define the ``TRAIN_DAYS`` and ``test_day`` days, **optionally** ``TOTAL_TIMESTEPS``

In [ ]:
TRAIN_DAYS = [0, 1, 2, 3]
TEST_DAY = 4
days_to_process = sorted(set(TRAIN_DAYS + [TEST_DAY]))
TOTAL_TIMESTEPS = 25000

# Set inference tick
TICK = 23

### 1. Preprocessing all `User Equipement` **(UE)** Data
Preprocess the raw UE data CSV files is performed to ensure compatibility with the RL Gym environment

In [ ]:
# 500 UEs x 24 ticks x 5 days = 60000 total data points
logger.info(f"--- Running UE Data Preprocessing for days: {days_to_process} ---")
preprocessor = UEDataPreprocessor(base_data_dir=BASE_DATA_DIR)
preprocessor.run(days=days_to_process)
logger.info("--- Preprocessing Step Finished ---")

### 2. Training core `bayesian_digital_twins` **(BDT)**
Call for the training of the Bayesian Digital Twin model, as well as the downloading of the resulting model

In [ ]:
# Set parameters
BDT_MODEL_ID = "bdt_for_energy_saving"
CONTAINER_NAME = "radp_dev-training-1"

In [ ]:
# Train bayesian_digital_twins from scratch
logger.info("--- Running BDT Training ---")
bdt_manager = BDTManager(
    topology_path=TOPOLOGY_PATH,
    training_data_path=TRAINING_DATA_PATH,
    model_path=BDT_MODEL_PATH
)

bdt_manager.train(model_id=BDT_MODEL_ID, container_name=CONTAINER_NAME)

logger.info("--- BDT Training Finished ---")

### 3. Train `RL Agent` for **Energy Savings** on The First 4 Days
- Initialize and train a `reinforcement learning` (RL) agent for energy-saving applications with an environment set up using `site configuration` & `UE` data, and a `BDT` model.
- Log monitors training process & saves checkpoints periodically

In [ ]:
# Intialize and train RL agent
logger.info(f"--- Running RL Training on Days: {TRAIN_DAYS} ---")

run_rl_training(
    bdt_model_path=BDT_MODEL_PATH,
    base_ue_data_dir=BASE_DATA_DIR,
    training_days=TRAIN_DAYS,
    topology_path=TOPOLOGY_PATH,
    config_path=CONFIG_PATH,
    rl_model_path=RL_MODEL_PATH,
    log_dir=RL_LOG_DIR,
    total_timesteps=TOTAL_TIMESTEPS
)

logger.info("--- RL Training Finished ---")

### 4. Inference
- Load `RL agent` and predicts the optimal cell configuration, including `tilt angles` and `on/off states`, for energy-saving applications. 
- Prediction is performed for a specified `time tick` (hour). 

In [ ]:
# Run inference on exising RL model
logger.info(f"--- Running Inference for Tick {TICK} ---")
run_rl_prediction(
    model_load_path=RL_MODEL_PATH,
    topology_path=TOPOLOGY_PATH,
    target_tick=TICK
)
logger.info("--- Inference Step Finished ---")

### 5. Visualization
Perform the visualization of network performance & compare baseline configuration to an RL-optimized configuration.


In [ ]:
# Generate visualization plots
logger.info(f"--- Running Visualization for Day {TEST_DAY}, Tick {TICK} ---")

try:
    visualizer = EnergySavingVisualizer(
        bdt_model_path=BDT_MODEL_PATH,
        rl_model_path=RL_MODEL_PATH,
        topology_path=TOPOLOGY_PATH,
        config_path=CONFIG_PATH,
        base_ue_data_dir=BASE_DATA_DIR
    )
    
    # Generate and save comparison plots for the baseline and RL-optimized scenarios.
    visualizer.generate_comparison_plots(
        day=TEST_DAY,
        tick=TICK,
        output_dir=PLOT_OUTPUT_DIR
    )
    logger.info("--- Visualization Step Finished ---")
    
except Exception as e:
    logger.exception(f"Visualization failed with error: {e}")

In [ ]:
image_path = os.path.join(PLOT_OUTPUT_DIR, f"energy_saving_comparison_day{TEST_DAY}_tick_{TICK}.png")

# Display plot image
display(Image(filename=image_path))